In [15]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import itertools as it

In [4]:
pric = [[1200, 1000, 950],
        [900, 800, 600],
        [500, 300, 200],
        [1400, 1300, 1150],
        [1100, 900, 750],
        [700, 400, 350],
        [1500, 900, 850],
        [820, 800, 500],
        [480, 470, 450]]

hdmn = [[25, 30, 40],
        [50, 40, 45],
        [50, 53, 65],
        [22, 45, 50],
        [45, 55, 75],
        [50, 60, 80],
        [45, 60, 75],
        [20, 40, 50],
        [55, 60, 75]]

fdmn = [[10, 15, 20],
        [20, 25, 35],
        [45, 55, 60],
        [20, 25, 35],
        [40, 42, 45],
        [50, 52, 63],
        [45, 50, 60],
        [45, 46, 47],
        [55, 56, 64],       
        [20, 25, 35],
        [42, 45, 46],
        [50, 52, 60],       
        [10, 40, 50],
        [50, 60, 80],
        [60, 65, 90],       
        [50, 55, 80],
        [20, 30, 50],
        [10, 40, 60],       
        [30, 35, 40],
        [40, 50, 55],
        [50, 60, 80],       
        [30, 40, 60],
        [10, 40, 45],
        [50, 60, 70],        
        [50, 70, 80],
        [40, 45, 60],
        [60, 65, 70]]

classes = ['1st', 'bus', 'eco']
periods = [1, 2, 3]
options = [1, 2, 3]
scenarios = [1, 2, 3]
ml2_index = pd.MultiIndex.from_tuples(it.product(periods, classes),
                                     names=['Period', 'Class'])
ml3_index = pd.MultiIndex.from_tuples(it.product(periods, scenarios, classes),
                                     names=['Period', 'Scenario', 'Class'])

P = pd.DataFrame(pric, columns=options, index=ml2_index)     #price options
Dh = pd.DataFrame(hdmn, columns=options, index=ml2_index)
Df = pd.DataFrame(fdmn, columns=options, index=ml3_index)

Q = dict(zip(scenarios, [0.1, 0.7, 0.2]))   # scenario probability
number_seats = dict(zip(classes, [37, 38, 47]))

plane_cost = 50000
n_planes = 6
weeks = [1, 2, 3]

In [ ]:
# if I ever want to implement identically to the model in the book
# vars
    # revenue
r1ich = model1.addVars(scenarios, classes, options, name='r3')
r2ijch = model1.addVars(scenarios, scenarios, classes, options, name='r3')
r3ijkch = model1.addVars(scenarios, scenarios, scenarios, classes, options, name='r3')

    # slack, surplus
uijkc = model1.addVars(scenarios, scenarios, scenarios, classes, name='u')
vijkc = model1.addVars(scenarios, scenarios, scenarios, classes, name='v')

# objective function
model1.setObjective(gp.quicksum(Q[i]*r1ich[i,c,h] 
                                for i in scenars for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*r2ijch[i,j,c,h] 
                                 for i in scenars for j in scenars for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*Q[k]*r3ijkch[i,j,k,c,h] 
                                 for i in scenars for j in scenars for k in scenars for c in classes for h in options)
                   - plane_cost*n)

# Solving Week 1

In [5]:
model1 = gp.Model('Yield Management - Week 1')

# add vars
    # price option chosen
p1ch = model1.addVars(classes, options, vtype=GRB.BINARY, name='p1')
p2ich = model1.addVars(scenarios, classes, options, vtype=GRB.BINARY, name='p2')
p3ijch = model1.addVars(scenarios, scenarios, classes, options, vtype=GRB.BINARY, name='p3')
    
    # sold tickets
s1ich = model1.addVars(scenarios, classes, options, name='s3')
s2ijch = model1.addVars(scenarios, scenarios, classes, options, name='s3')
s3ijkch = model1.addVars(scenarios, scenarios, scenarios, classes, options, name='s3')

    # number of planes necessary
n = model1.addVar(ub=n_planes, vtype=GRB.INTEGER, name='n')

# objective function
model1.setObjective((gp.quicksum(Q[i]*P[h][1,c]*p1ch[c,h]*s1ich[i,c,h]  
                                for i in scenarios for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*P[h][2,c]*p2ich[i,c,h]*s2ijch[i,j,c,h]  
                                 for i in scenarios for j in scenarios for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*Q[k]*P[h][3,c]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h]
                                 for i in scenarios for j in scenarios for k in scenarios for c in classes for h in options)
                   - plane_cost*n), GRB.MAXIMIZE)

# add constraints 
    # WEEK 1 choose only one price option
model1.addConstrs((gp.quicksum(p1ch[c,h] for h in options) == 1 
                               for c in classes),
                  name='w1_p_options')

    # WEEK 1 sell less than demand
model1.addConstrs((s1ich[i,c,h] <= Df[h][1,i,c]*p1ch[c,h] 
                  for i in scenarios for c in classes for h in options),
                  name='w1_sales')

    # WEEK 2 choose only one price option
model1.addConstrs((gp.quicksum(p2ich[i,c,h] for h in options) == 1 
                               for i in scenarios for c in classes),
                  name='w2_p_options')

    # WEEK 2 sell less than demand
model1.addConstrs((s2ijch[i,j,c,h] <= Df[h][2,j,c]*p2ich[i,c,h] 
                  for i in scenarios for j in scenarios for c in classes for h in options),
                  name='w2_sales')

    # WEEK 3 choose only one price option
model1.addConstrs((gp.quicksum(p3ijch[i,j,c,h] for h in options) == 1 
                               for i in scenarios for j in scenarios for c in classes),
                  name='w3_p_options')

    # WEEK 3 sell less than demand
model1.addConstrs((s3ijkch[i,j,k,c,h] <= Df[h][3,k,c]*p3ijch[i,j,c,h] 
                  for i in scenarios for j in scenarios for k in scenarios for c in classes for h in options),
                  name='w3_sales')

    # seat capacity constraints
model1.addConstrs((gp.quicksum(s1ich[i,c,h] for h in options)
                  + gp.quicksum(s2ijch[i,j,c,h] for h in options)
                  + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)
                  <= number_seats[c]*n
                  for i in scenarios for j in scenarios for k in scenarios for c in classes),
                  name='seat_cap')

model1.update()
model1.write('Yield Management Week 1.lp')
model1.Params.nonConvex=2
model1.optimize()

w1_solution = {(c,h): abs(p1ch[c,h].x) for c,h in p1ch.keys()}

Using license file C:\Users\Nara\gurobi.lic
Academic license - for non-commercial use only - expires 2021-05-08


# Solving Week 2

Using the results found for Week 1

In [22]:
model2 = gp.Model('Yield Management - Week 2')

# add vars
    # price option chosen
p1ch = model2.addVars(classes, options, vtype=GRB.BINARY, name='p1')
p2ich = model2.addVars(scenarios, classes, options, vtype=GRB.BINARY, name='p2')
p3ijch = model2.addVars(scenarios, scenarios, classes, options, vtype=GRB.BINARY, name='p3')
    
    # sold tickets
s1ich = model2.addVars(scenarios, classes, options, name='s3')
s2ijch = model2.addVars(scenarios, scenarios, classes, options, name='s3')
s3ijkch = model2.addVars(scenarios, scenarios, scenarios, classes, options, name='s3')

    # number of planes necessary
n = model2.addVar(ub=n_planes, vtype=GRB.INTEGER, name='n')

# objective function
model2.setObjective((gp.quicksum(Q[i]*P[h][1,c]*p1ch[c,h]*s1ich[i,c,h]  
                                for i in scenarios for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*P[h][2,c]*p2ich[i,c,h]*s2ijch[i,j,c,h]  
                                 for i in scenarios for j in scenarios for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*Q[k]*P[h][3,c]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h]
                                 for i in scenarios for j in scenarios for k in scenarios for c in classes for h in options)
                   - plane_cost*n), GRB.MAXIMIZE)

# add constraints 
    # WEEK 1 choose only one price option
model2.addConstrs((gp.quicksum(p1ch[c,h] for h in options) == 1 
                               for c in classes),
                  name='w1_p_options')

    # WEEK 1 sell less than demand. REMEMBER: now it's the actual demand for week 1!
model2.addConstrs((s1ich[i,c,h] <= Dh[h][1,c]*p1ch[c,h] 
                  for i in scenarios for c in classes for h in options),
                  name='w1_sales')

    # WEEK 2 choose only one price option
model2.addConstrs((gp.quicksum(p2ich[i,c,h] for h in options) == 1 
                               for i in scenarios for c in classes),
                  name='w2_p_options')

    # WEEK 2 sell less than demand
model2.addConstrs((s2ijch[i,j,c,h] <= Df[h][2,j,c]*p2ich[i,c,h] 
                  for i in scenarios for j in scenarios for c in classes for h in options),
                  name='w2_sales')

    # WEEK 3 choose only one price option
model2.addConstrs((gp.quicksum(p3ijch[i,j,c,h] for h in options) == 1 
                               for i in scenarios for j in scenarios for c in classes),
                  name='w3_p_options')

    # WEEK 3 sell less than demand
model2.addConstrs((s3ijkch[i,j,k,c,h] <= Df[h][3,k,c]*p3ijch[i,j,c,h] 
                  for i in scenarios for j in scenarios for k in scenarios for c in classes for h in options),
                  name='w3_sales')

    # seat capacity constraints
model2.addConstrs((gp.quicksum(s1ich[i,c,h] for h in options)
                  + gp.quicksum(s2ijch[i,j,c,h] for h in options)
                  + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)
                  <= number_seats[c]*n
                  for i in scenarios for j in scenarios for k in scenarios for c in classes),
                  name='seat_cap')

# setting up the variables with the results found in Week 1
for c in classes:
    for h in options:
        p1ch[c,h].lb = w1_solution[c,h]

        
model2.update()
model2.write('Yield Management Week 2.lp')
model2.Params.nonConvex=2
model2.optimize()

w2_solution = {(i,c,h): abs(p2ich[i,c,h].x) for i,c,h in p2ich.keys()}

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1629 nonzeros
Model fingerprint: 0xb54aa103
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -300000.0000
Presolve removed 30 rows and 27 columns
Presolve time: 0.00s
Presolved: 765 rows, 766 columns, 2376 nonzeros
Variable types: 657 continuous, 109 integer (108 binary)

Root relaxation: objective -1.743903e+05, 456 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

# Solving Week 3
Using the solutions found in Week 1 and 2

In [25]:
model3 = gp.Model('Yield Management - Week 3')

# add vars
    # price option chosen
p1ch = model3.addVars(classes, options, vtype=GRB.BINARY, name='p1')
p2ich = model3.addVars(scenarios, classes, options, vtype=GRB.BINARY, name='p2')
p3ijch = model3.addVars(scenarios, scenarios, classes, options, vtype=GRB.BINARY, name='p3')
    
    # sold tickets
s1ich = model3.addVars(scenarios, classes, options, name='s3')
s2ijch = model3.addVars(scenarios, scenarios, classes, options, name='s3')
s3ijkch = model3.addVars(scenarios, scenarios, scenarios, classes, options, name='s3')

    # number of planes necessary
n = model3.addVar(ub=n_planes, vtype=GRB.INTEGER, name='n')

# objective function
model3.setObjective((gp.quicksum(Q[i]*P[h][1,c]*p1ch[c,h]*s1ich[i,c,h]  
                                for i in scenarios for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*P[h][2,c]*p2ich[i,c,h]*s2ijch[i,j,c,h]  
                                 for i in scenarios for j in scenarios for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*Q[k]*P[h][3,c]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h]
                                 for i in scenarios for j in scenarios for k in scenarios for c in classes for h in options)
                   - plane_cost*n), GRB.MAXIMIZE)

# add constraints 
    # WEEK 1 choose only one price option
model3.addConstrs((gp.quicksum(p1ch[c,h] for h in options) == 1 
                               for c in classes),
                  name='w1_p_options')

    # WEEK 1 sell less than demand. REMEMBER: now it's the actual demand for week 1!
model3.addConstrs((s1ich[i,c,h] <= Dh[h][1,c]*p1ch[c,h] 
                  for i in scenarios for c in classes for h in options),
                  name='w1_sales')

    # WEEK 2 choose only one price option
model3.addConstrs((gp.quicksum(p2ich[i,c,h] for h in options) == 1 
                               for i in scenarios for c in classes),
                  name='w2_p_options')

    # WEEK 2 sell less than demand. REMEMBER: now it's the actual demand for week 2!
model3.addConstrs((s2ijch[i,j,c,h] <= Dh[h][2,c]*p2ich[i,c,h] 
                  for i in scenarios for j in scenarios for c in classes for h in options),
                  name='w2_sales')

    # WEEK 3 choose only one price option
model3.addConstrs((gp.quicksum(p3ijch[i,j,c,h] for h in options) == 1 
                               for i in scenarios for j in scenarios for c in classes),
                  name='w3_p_options')

    # WEEK 3 sell less than demand
model3.addConstrs((s3ijkch[i,j,k,c,h] <= Df[h][3,k,c]*p3ijch[i,j,c,h] 
                  for i in scenarios for j in scenarios for k in scenarios for c in classes for h in options),
                  name='w3_sales')

    # seat capacity constraints
model3.addConstrs((gp.quicksum(s1ich[i,c,h] for h in options)
                  + gp.quicksum(s2ijch[i,j,c,h] for h in options)
                  + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)
                  <= number_seats[c]*n
                  for i in scenarios for j in scenarios for k in scenarios for c in classes),
                  name='seat_cap')

# setting up the variables with the results found in Week 1 and 2
for c in classes:
    for h in options:
        for i in scenarios:
            p1ch[c,h].lb = w1_solution[c,h]
            p2ich[i,c,h].lb = w2_solution[i,c,h]
        
model3.update()
model3.write('Yield Management Week 3.lp')
model3.Params.nonConvex=2
model3.optimize()

w3_solution = {(i,j,c,h): abs(p3ijch[i,j,c,h].x) for i,j,c,h in p3ijch.keys()}

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1629 nonzeros
Model fingerprint: 0xa9c38f0c
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -300000.0000
Presolve removed 120 rows and 108 columns
Presolve time: 0.00s
Presolved: 594 rows, 604 columns, 1782 nonzeros
Variable types: 522 continuous, 82 integer (81 binary)

Root relaxation: objective -1.786023e+05, 389 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

# Solving at Take-off
Using the results found for each Week

In [27]:
modelt = gp.Model('Yield Management - Take-off')

# add vars
    # price option chosen
p1ch = modelt.addVars(classes, options, vtype=GRB.BINARY, name='p1')
p2ich = modelt.addVars(scenarios, classes, options, vtype=GRB.BINARY, name='p2')
p3ijch = modelt.addVars(scenarios, scenarios, classes, options, vtype=GRB.BINARY, name='p3')
    
    # sold tickets
s1ich = modelt.addVars(scenarios, classes, options, name='s3')
s2ijch = modelt.addVars(scenarios, scenarios, classes, options, name='s3')
s3ijkch = modelt.addVars(scenarios, scenarios, scenarios, classes, options, name='s3')

    # number of planes necessary
n = modelt.addVar(ub=n_planes, vtype=GRB.INTEGER, name='n')

# objective function
modelt.setObjective((gp.quicksum(Q[i]*P[h][1,c]*p1ch[c,h]*s1ich[i,c,h]  
                                for i in scenarios for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*P[h][2,c]*p2ich[i,c,h]*s2ijch[i,j,c,h]  
                                 for i in scenarios for j in scenarios for c in classes for h in options)
                   + gp.quicksum(Q[i]*Q[j]*Q[k]*P[h][3,c]*p3ijch[i,j,c,h]*s3ijkch[i,j,k,c,h]
                                 for i in scenarios for j in scenarios for k in scenarios for c in classes for h in options)
                   - plane_cost*n), GRB.MAXIMIZE)

# add constraints 
    # WEEK 1 choose only one price option
modelt.addConstrs((gp.quicksum(p1ch[c,h] for h in options) == 1 
                               for c in classes),
                  name='w1_p_options')

    # WEEK 1 sell less than demand. REMEMBER: now it's the actual demand for week 1!
modelt.addConstrs((s1ich[i,c,h] <= Dh[h][1,c]*p1ch[c,h] 
                  for i in scenarios for c in classes for h in options),
                  name='w1_sales')

    # WEEK 2 choose only one price option
modelt.addConstrs((gp.quicksum(p2ich[i,c,h] for h in options) == 1 
                               for i in scenarios for c in classes),
                  name='w2_p_options')

    # WEEK 2 sell less than demand. REMEMBER: now it's the actual demand for week 2!
modelt.addConstrs((s2ijch[i,j,c,h] <= Dh[h][2,c]*p2ich[i,c,h] 
                  for i in scenarios for j in scenarios for c in classes for h in options),
                  name='w2_sales')

    # WEEK 3 choose only one price option
modelt.addConstrs((gp.quicksum(p3ijch[i,j,c,h] for h in options) == 1 
                               for i in scenarios for j in scenarios for c in classes),
                  name='w3_p_options')

    # WEEK 3 sell less than demand. REMEMBER: now it's the actual demand for week 3!
modelt.addConstrs((s3ijkch[i,j,k,c,h] <= Dh[h][3,c]*p3ijch[i,j,c,h] 
                  for i in scenarios for j in scenarios for k in scenarios for c in classes for h in options),
                  name='w3_sales')

    # seat capacity constraints
modelt.addConstrs((gp.quicksum(s1ich[i,c,h] for h in options)
                  + gp.quicksum(s2ijch[i,j,c,h] for h in options)
                  + gp.quicksum(s3ijkch[i,j,k,c,h] for h in options)
                  <= number_seats[c]*n
                  for i in scenarios for j in scenarios for k in scenarios for c in classes),
                  name='seat_cap')

# setting up the variables with the results found in Week 1, and 3
for c in classes:
    for h in options:
        for i in scenarios:
            for j in scenarios:
                p1ch[c,h].lb = w1_solution[c,h]
                p2ich[i,c,h].lb = w2_solution[i,c,h]
                p3ijch[i,j,c,h].lb = w3_solution[i,j,c,h]
        
modelt.update()
modelt.write('Yield Management Take-off.lp')
modelt.Params.nonConvex=2
modelt.optimize()

t_solution = {(i,j,c,h): abs(p3ijch[i,j,c,h].x) for i,j,c,h in p3ijch.keys()}

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 471 rows, 469 columns and 1629 nonzeros
Model fingerprint: 0x3c30153a
Model has 351 quadratic objective terms
Variable types: 351 continuous, 118 integer (117 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+04, 5e+04]
  QObjective range [9e-01, 2e+03]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 470 rows and 458 columns
Presolve time: 0.00s
Presolved: 1 rows, 11 columns, 11 nonzeros
Variable types: 10 continuous, 1 integer (0 binary)

Root relaxation: objective 1.952617e+05, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestB

In [54]:
print('Total profit of ${:,.2f}'.format(modelt.objVal))
print('Use {:.0f} planes'.format(n.x))

sol_index = pd.MultiIndex.from_tuples(it.product(periods, classes),
                                      names=['Period', 'Class'])
report = pd.DataFrame([], columns=['Seats', 'Price Option'], index=sol_index)

for i,j,k,c,h in s3ijkch.keys():
    for p in periods:
        if p == 3:
            if s3ijkch[i,j,k,c,h].x>0:
                report['Seats'][p,c] = s3ijkch[i,j,k,c,h].x
            if p3ijch[i,j,c,h].x>0.9:
                report['Price Option'][p,c] = P[h][p,c]
        if p == 2:
            if s2ijch[i,j,c,h].x>0:
                report['Seats'][p,c] = s2ijch[i,j,c,h].x
            if p2ich[i,c,h].x>0.9:
                report['Price Option'][p,c] = P[h][p,c]
        if p == 1:
            if s1ich[i,c,h].x>0:
                report['Seats'][p,c] = s1ich[i,c,h].x
            if p1ch[c,h].x>0.9:
                report['Price Option'][p,c] = P[h][p,c]
            
report

Total profit of $184,030.00
Use 3 planes


Seats Price Option
Period Class                   
1      1st    25.0         1200
       bus    50.0          900
       eco    50.0          500
2      1st    41.0         1150
       bus    45.0         1100
       eco    50.0          700
3      1st    45.0         1500
       bus    19.0          800
       eco    41.0          480

In [47]:
s2ijch

{(1, 1, '1st', 1): <gurobi.Var s3[1,1,1st,1] (value 0.0)>,
 (1, 1, '1st', 2): <gurobi.Var s3[1,1,1st,2] (value 0.0)>,
 (1, 1, '1st', 3): <gurobi.Var s3[1,1,1st,3] (value 41.0)>,
 (1, 1, 'bus', 1): <gurobi.Var s3[1,1,bus,1] (value 45.0)>,
 (1, 1, 'bus', 2): <gurobi.Var s3[1,1,bus,2] (value 0.0)>,
 (1, 1, 'bus', 3): <gurobi.Var s3[1,1,bus,3] (value 0.0)>,
 (1, 1, 'eco', 1): <gurobi.Var s3[1,1,eco,1] (value 50.0)>,
 (1, 1, 'eco', 2): <gurobi.Var s3[1,1,eco,2] (value 0.0)>,
 (1, 1, 'eco', 3): <gurobi.Var s3[1,1,eco,3] (value 0.0)>,
 (1, 2, '1st', 1): <gurobi.Var s3[1,2,1st,1] (value 0.0)>,
 (1, 2, '1st', 2): <gurobi.Var s3[1,2,1st,2] (value 0.0)>,
 (1, 2, '1st', 3): <gurobi.Var s3[1,2,1st,3] (value 41.0)>,
 (1, 2, 'bus', 1): <gurobi.Var s3[1,2,bus,1] (value 45.0)>,
 (1, 2, 'bus', 2): <gurobi.Var s3[1,2,bus,2] (value 0.0)>,
 (1, 2, 'bus', 3): <gurobi.Var s3[1,2,bus,3] (value 0.0)>,
 (1, 2, 'eco', 1): <gurobi.Var s3[1,2,eco,1] (value 50.0)>,
 (1, 2, 'eco', 2): <gurobi.Var s3[1,2,eco,2] (valu

In [50]:
for i,j,k,c,h in s3ijkch.keys():
    if s3ijkch[i,j,k,c,h].x>0:
        print(s3ijkch[i,j,k,c,h])

<gurobi.Var s3[1,1,1,1st,1] (value 45.0)>
<gurobi.Var s3[1,1,1,bus,2] (value 19.0)>
<gurobi.Var s3[1,1,1,eco,1] (value 41.0)>
<gurobi.Var s3[1,1,2,1st,1] (value 45.0)>
<gurobi.Var s3[1,1,2,bus,2] (value 19.0)>
<gurobi.Var s3[1,1,2,eco,1] (value 41.0)>
<gurobi.Var s3[1,1,3,1st,1] (value 45.0)>
<gurobi.Var s3[1,1,3,bus,2] (value 19.0)>
<gurobi.Var s3[1,1,3,eco,1] (value 41.0)>
<gurobi.Var s3[1,2,1,1st,1] (value 45.0)>
<gurobi.Var s3[1,2,1,bus,2] (value 19.0)>
<gurobi.Var s3[1,2,1,eco,1] (value 41.0)>
<gurobi.Var s3[1,2,2,1st,1] (value 45.0)>
<gurobi.Var s3[1,2,2,bus,2] (value 19.0)>
<gurobi.Var s3[1,2,2,eco,1] (value 41.0)>
<gurobi.Var s3[1,2,3,1st,1] (value 45.0)>
<gurobi.Var s3[1,2,3,bus,2] (value 19.0)>
<gurobi.Var s3[1,2,3,eco,1] (value 41.0)>
<gurobi.Var s3[1,3,1,1st,1] (value 45.0)>
<gurobi.Var s3[1,3,1,bus,2] (value 19.0)>
<gurobi.Var s3[1,3,1,eco,1] (value 41.0)>
<gurobi.Var s3[1,3,2,1st,1] (value 45.0)>
<gurobi.Var s3[1,3,2,bus,2] (value 19.0)>
<gurobi.Var s3[1,3,2,eco,1] (value